In [1]:
import zipfile
with zipfile.ZipFile("homework.zip","r") as zip_ref:
    zip_ref.extractall()

In [2]:
import pandas as pd
import glob
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

In [3]:
data_walking = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/walking/*.csv"))
data_running = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/running/*.csv"))
data_idle = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/idle/*.csv"))
data_stairs = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/stairs/*.csv"))

In [5]:
data_walking['activity'] = 'walking'
data_running['activity'] = 'running'
data_idle['activity'] = 'idle'
data_stairs['activity'] = 'stairs'

data = pd.concat([data_walking, data_running, data_idle, data_stairs])

Розрахунок фічів

In [6]:
data['accelerometer_X_mean'] = data.groupby('activity')['accelerometer_X'].transform('mean')
data['accelerometer_X_std'] = data.groupby('activity')['accelerometer_X'].transform('std')
data['accelerometer_X_max'] = data.groupby('activity')['accelerometer_X'].transform('max')
data['accelerometer_X_min'] = data.groupby('activity')['accelerometer_X'].transform('min')
data['accelerometer_X_range'] = data['accelerometer_X_max'] - data['accelerometer_X_min']

data['accelerometer_Z_mean'] = data.groupby('activity')['accelerometer_Z'].transform('mean')
data['accelerometer_Z_std'] = data.groupby('activity')['accelerometer_Z'].transform('std')
data['accelerometer_Z_max'] = data.groupby('activity')['accelerometer_Z'].transform('max')
data['accelerometer_Z_min'] = data.groupby('activity')['accelerometer_Z'].transform('min')
data['accelerometer_Z_range'] = data['accelerometer_Z_max'] - data['accelerometer_Z_min']

data['accelerometer_Y_mean'] = data.groupby('activity')['accelerometer_Y'].transform('mean')
data['accelerometer_Y_std'] = data.groupby('activity')['accelerometer_Y'].transform('std')
data['accelerometer_Y_max'] = data.groupby('activity')['accelerometer_Y'].transform('max')
data['accelerometer_Y_min'] = data.groupby('activity')['accelerometer_Y'].transform('min')
data['accelerometer_Y_range'] = data['accelerometer_Y_max'] - data['accelerometer_Y_min']

In [7]:
data['accelerometer_mean_combined'] = data[['accelerometer_X_mean', 'accelerometer_Y_mean', 'accelerometer_Z_mean']].mean(axis=1)
data['accelerometer_std_combined'] = data[['accelerometer_X_std', 'accelerometer_Y_std', 'accelerometer_Z_std']].std(axis=1)
data['accelerometer_max_combined'] = data[['accelerometer_X_max', 'accelerometer_Y_max', 'accelerometer_Z_max']].max(axis=1)
data['accelerometer_min_combined'] = data[['accelerometer_X_min', 'accelerometer_Y_min', 'accelerometer_Z_min']].min(axis=1)
data['accelerometer_range_combined'] = data[['accelerometer_X_range', 'accelerometer_Y_range', 'accelerometer_Z_range']].max(axis=1)

Виділяємо ознаки та цільову змінну

In [9]:
features = data[['accelerometer_X_mean', 'accelerometer_X_std', 'accelerometer_X_max', 'accelerometer_X_min', 'accelerometer_X_range',
                 'accelerometer_Z_mean', 'accelerometer_Z_std', 'accelerometer_Z_max', 'accelerometer_Z_min', 'accelerometer_Z_range',
                 'accelerometer_Y_mean', 'accelerometer_Y_std', 'accelerometer_Y_max', 'accelerometer_Y_min', 'accelerometer_Y_range',
                 'accelerometer_mean_combined', 'accelerometer_std_combined', 'accelerometer_max_combined', 'accelerometer_min_combined', 'accelerometer_range_combined']]

target = data['activity']

Розбиваємо датасет на навчальний і тренувальний

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Навчання моделі SVM

In [11]:
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

SVC()

# Навчання моделі Випадкового лісу

In [12]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

# Прогнозування на тестовому наборі

In [13]:
svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

# Оцінка продуктивності моделей

In [14]:
svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Точність моделі SVM:", svm_accuracy)
print("Точність моделі Випадкового лісу:", rf_accuracy)

Точність моделі SVM: 1.0
Точність моделі Випадкового лісу: 1.0
